# Structure Server - Comprehensive Test Suite

This notebook tests all MCP tools in `structure_server.py`:

1. **fetch_molecules** - Fetch structures from PDB/AlphaFold/PDB-REDO
2. **inspect_molecules** - Inspect structure files to analyze chains and molecules
3. **split_molecules** - Split multi-chain structures into individual chains
4. **clean_protein** - Clean protein structures for MD simulation
5. **clean_ligand** - Clean and prepare ligands using SMILES template matching
6. **run_antechamber_robust** - GAFF2 parameterization with AM1-BCC charges
7. **prepare_complex** - Complete workflow (split + clean + parameterize)

Each tool is tested for:
- Normal operation
- Edge cases
- Error handling (LLM-friendly error responses)
- Boltz-2 predicted structures (computational models)
- Ligand preparation and force field generation


In [43]:
# Setup
import sys
sys.path.insert(0, '..')

from pathlib import Path
import json
import importlib
import asyncio

# For running async functions in notebook
import nest_asyncio
nest_asyncio.apply()

print("Setup complete")


Setup complete


In [44]:
# Check dependencies
print("Checking dependencies...\n")

deps = {
    "gemmi": "Structure parsing (mmCIF/PDB)",
    "pdbfixer": "Protein structure cleaning",
    "openmm": "Molecular simulation",
    "httpx": "Async HTTP client",
    "rdkit": "Ligand processing and charge estimation"
}

for module, desc in deps.items():
    try:
        __import__(module)
        print(f"✓ {module}: {desc}")
    except ImportError:
        print(f"✗ {module}: {desc} (NOT INSTALLED)")

# Check external tools
print("\nChecking external tools...")
from common.base import BaseToolWrapper

tools = {
    "pdb4amber": "Amber naming conventions",
    "antechamber": "GAFF2 parameterization",
    "parmchk2": "Missing parameter generation",
    "obabel": "Format conversion"
}

for tool, desc in tools.items():
    wrapper = BaseToolWrapper(tool, conda_env="mcp-md")
    print(f"{'✓' if wrapper.is_available() else '✗'} {tool} ({desc})")


Checking dependencies...

✓ gemmi: Structure parsing (mmCIF/PDB)
✓ pdbfixer: Protein structure cleaning
✓ openmm: Molecular simulation
✓ httpx: Async HTTP client
✓ rdkit: Ligand processing and charge estimation

Checking external tools...
✓ pdb4amber (Amber naming conventions)
✓ antechamber (GAFF2 parameterization)
✓ parmchk2 (Missing parameter generation)
✓ obabel (Format conversion)


In [45]:
# Import and reload the structure server module
import servers.structure_server as structure_module
importlib.reload(structure_module)

# Import tools directly
from servers.structure_server import (
    fetch_molecules,
    inspect_molecules,
    split_molecules,
    clean_protein,
    clean_ligand,
    run_antechamber_robust,
    prepare_complex
)

print("Structure server tools imported successfully")


[12/05/25 19:09:35] WARNING  FASPR not found in PATH                                                     ]8;id=206735;file:///Users/yasu/tmp/mcp-md/notebooks/../common/base.py\base.py]8;;\:]8;id=844264;file:///Users/yasu/tmp/mcp-md/notebooks/../common/base.py#94\94]8;;\

Structure server tools imported successfully


In [46]:
# Helper function to display results nicely
def show_result(result: dict, title: str = "Result"):
    """Display result dictionary with formatting"""
    print(f"\n{'='*60}")
    print(f" {title}")
    print(f"{'='*60}")
    
    # Check success status
    if result.get('success'):
        print("\n✓ SUCCESS")
    else:
        print("\n✗ FAILED")
    
    # Show errors if any
    if result.get('errors'):
        print("\nErrors:")
        for err in result['errors']:
            print(f"  - {err}")
    
    # Show warnings if any
    if result.get('warnings'):
        print("\nWarnings:")
        for warn in result['warnings']:
            print(f"  - {warn}")
    
    # Show key fields
    skip_keys = {'success', 'errors', 'warnings', 'operations'}
    print("\nDetails:")
    for k, v in result.items():
        if k not in skip_keys:
            if isinstance(v, (dict, list)) and len(str(v)) > 100:
                print(f"  {k}: [complex data, {len(v) if isinstance(v, list) else 'dict'}]")
            else:
                print(f"  {k}: {v}")
    
    # Show operations if present
    if result.get('operations'):
        print("\nOperations:")
        for op in result['operations']:
            status_icon = "✓" if op.get('status') in ['success', 'detected', 'added', 'replaced'] else "○"
            print(f"  {status_icon} {op.get('step')}: {op.get('status')} - {op.get('details', '')[:60]}")

print("Helper function defined")


Helper function defined


---
## Test 1: fetch_molecules

Test fetching structures from different sources.


In [47]:
# Test 1.1: Fetch from PDB (small protein: 1CRN - crambin)
print("Test 1.1: Fetch 1CRN from PDB")

result = asyncio.run(fetch_molecules("1CRN", source="pdb"))
show_result(result, "Fetch 1CRN from PDB")

# Verify file exists
if result['success'] and result['file_path']:
    print(f"\nFile size: {Path(result['file_path']).stat().st_size} bytes")


2025-12-05 19:09:35,894 - servers.structure_server - INFO - Fetching 1CRN from pdb


Test 1.1: Fetch 1CRN from PDB


                    INFO     Fetching 1CRN from pdb                                         ]8;id=123566;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=418010;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#675\675]8;;\

[12/05/25 19:09:36] INFO     HTTP Request: GET https://files.rcsb.org/download/1CRN.cif "HTTP/1.1   ]8;id=393859;file:///Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=857712;file:///Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/httpx/_client.py#1740\1740]8;;\
                             200 OK"                                                                               

2025-12-05 19:09:36,324 - servers.structure_server - INFO - Downloaded 1CRN to output/1CRN.cif


                    INFO     Downloaded 1CRN to output/1CRN.cif                             ]8;id=40148;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=412229;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#746\746]8;;\

2025-12-05 19:09:36,330 - servers.structure_server - INFO - Successfully fetched 1CRN: 327 atoms, chains: ['A']


                    INFO     Successfully fetched 1CRN: 327 atoms, chains: ['A']            ]8;id=276350;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=601443;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#784\784]8;;\


 Fetch 1CRN from PDB

✓ SUCCESS

Details:
  pdb_id: 1CRN
  source: pdb
  file_path: output/1CRN.cif
  file_format: cif
  num_atoms: 327
  chains: ['A']

File size: 69506 bytes


In [48]:
# Test 1.2: Fetch non-existent PDB ID (error handling)
print("Test 1.2: Fetch non-existent PDB ID")

result = asyncio.run(fetch_molecules("XXXX", source="pdb"))
show_result(result, "Fetch Invalid PDB ID")

# Check that error handling is LLM-friendly
assert not result['success'], "Should fail for invalid PDB ID"
assert len(result['errors']) > 0, "Should have error messages"
print("\n✓ Error handling works correctly")


2025-12-05 19:09:36,338 - servers.structure_server - INFO - Fetching XXXX from pdb


Test 1.2: Fetch non-existent PDB ID


                    INFO     Fetching XXXX from pdb                                         ]8;id=109184;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=255855;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#675\675]8;;\

                    INFO     HTTP Request: GET https://files.rcsb.org/download/XXXX.cif "HTTP/1.1   ]8;id=93696;file:///Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=88466;file:///Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/httpx/_client.py#1740\1740]8;;\
                             404 Not Found"                                                                        

[12/05/25 19:09:37] INFO     HTTP Request: GET https://files.rcsb.org/download/XXXX.pdb "HTTP/1.1   ]8;id=464251;file:///Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=243233;file:///Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/httpx/_client.py#1740\1740]8;;\
                             404 Not Found"                                                                        


 Fetch Invalid PDB ID

✗ FAILED

Errors:
  - Structure not found: XXXX (HTTP 404)
  - Hint: Verify the PDB ID is correct. Try searching at https://www.rcsb.org/

Warnings:
  - mmCIF not available, falling back to PDB format

Details:
  pdb_id: XXXX
  source: pdb
  file_path: None
  file_format: None
  num_atoms: 0
  chains: []

✓ Error handling works correctly


---
## Test 2: inspect_molecules

Test inspecting structure files to analyze chains and molecular composition.


In [49]:
# Test 2.1: Inspect 1AKE (homodimer with ligand)
print("Test 2.1: Inspect 1AKE structure")

# First fetch 1AKE
fetch_result = asyncio.run(fetch_molecules("1AKE", source="pdb"))
if fetch_result['success']:
    result = inspect_molecules(fetch_result['file_path'])
    show_result(result, "Inspect 1AKE")
    
    # Show detailed chain information
    if result['success']:
        print("\n--- Header Information ---")
        for k, v in result.get('header', {}).items():
            print(f"  {k}: {v}")
        
        print("\n--- Entities (from header) ---")
        for entity in result.get('entities', []):
            print(f"  Entity {entity['entity_id']}: {entity.get('name') or '(no name)'}")
            print(f"    Type: {entity['entity_type']}, Polymer: {entity.get('polymer_type')}")
            print(f"    Chains: {entity['chain_ids']}")
        
        print("\n--- Chain Summary ---")
        summary = result.get('summary', {})
        print(f"  Proteins: {summary.get('num_protein_chains', 0)} chains {summary.get('protein_chain_ids', [])}")
        print(f"  Ligands: {summary.get('num_ligand_chains', 0)} chains {summary.get('ligand_chain_ids', [])}")
        print(f"  Waters: {summary.get('num_water_chains', 0)} chains {summary.get('water_chain_ids', [])}")
        print(f"  Ions: {summary.get('num_ion_chains', 0)} chains {summary.get('ion_chain_ids', [])}")
        
        print("\n--- Chains Detail ---")
        for chain in result.get('chains', []):
            print(f"  Chain {chain['chain_id']} ({chain['author_chain']}): {chain['chain_type']}")
            print(f"    Entity: {chain.get('entity_name') or chain.get('entity_id') or 'N/A'}")
            print(f"    Residues: {chain['num_residues']}, Atoms: {chain['num_atoms']}")
            if chain.get('sequence'):
                seq = chain['sequence']
                print(f"    Sequence: {seq[:50]}{'...' if len(seq) > 50 else ''}")
else:
    print("Failed to fetch 1AKE for inspect test")


2025-12-05 19:09:37,624 - servers.structure_server - INFO - Fetching 1AKE from pdb


Test 2.1: Inspect 1AKE structure


                    INFO     Fetching 1AKE from pdb                                         ]8;id=535201;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=697428;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#675\675]8;;\

                    INFO     HTTP Request: GET https://files.rcsb.org/download/1AKE.cif "HTTP/1.1   ]8;id=531440;file:///Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=146962;file:///Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/httpx/_client.py#1740\1740]8;;\
                             200 OK"                                                                               

2025-12-05 19:09:37,664 - servers.structure_server - INFO - Downloaded 1AKE to output/1AKE.cif


...

2025-12-05 19:09:37,713 - servers.structure_server - INFO - Successfully inspected structure: 6 chains found


                    INFO     Successfully inspected structure: 6 chains found              ]8;id=372188;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=963672;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1116\1116]8;;\

2025-12-05 19:09:37,714 - servers.structure_server - INFO -   Proteins: 2, Ligands: 2, Waters: 2, Ions: 0


                    INFO       Proteins: 2, Ligands: 2, Waters: 2, Ions: 0                 ]8;id=431444;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=336950;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1117\1117]8;;\


 Inspect 1AKE

✓ SUCCESS

Details:
  source_file: output/1AKE.cif
  file_format: cif
  header: [complex data, dict]
  entities: [complex data, 3]
  num_models: 1
  chains: [complex data, 6]
  summary: [complex data, dict]

--- Header Information ---
  pdb_id: 1AKE
  title: STRUCTURE OF THE COMPLEX BETWEEN ADENYLATE KINASE FROM ESCHERICHIA COLI AND THE INHIBITOR AP5A REFINED AT 1.9 ANGSTROMS RESOLUTION: A MODEL FOR A CATALYTIC TRANSITION STATE
  resolution: 2.0
  spacegroup: P 21 2 21
  experiment_method: X-RAY DIFFRACTION

--- Entities (from header) ---
  Entity 1: (no name)
    Type: polymer, Polymer: PeptideL
    Chains: ['A', 'B']
  Entity 2: (no name)
    Type: nonpolymer, Polymer: None
    Chains: ['C', 'D']
  Entity 3: (no name)
    Type: water, Polymer: None
    Chains: ['E', 'F']

--- Chain Summary ---
  Proteins: 2 chains ['A', 'B']
  Ligands: 2 chains ['C', 'D']
  Waters: 2 chains ['E', 'F']
  Ions: 0 chains []

--- Chains Detail ---
  Chain A (A): protein
    Entity: 1
    

In [50]:
# Test 2.2: Inspect Boltz-2 predicted structure (computational model)
print("Test 2.2: Inspect Boltz-2 predicted structure")

boltz_cif = "boltz_results_ligand/predictions/ligand/ligand_model_0.cif"
result = inspect_molecules(boltz_cif)
show_result(result, "Inspect Boltz-2 Prediction")

# Show detailed information for AI-generated structure
if result['success']:
    print("\n--- Header Information ---")
    for k, v in result.get('header', {}).items():
        print(f"  {k}: {v}")
    
    print("\n--- Entities ---")
    for entity in result.get('entities', []):
        print(f"  Entity {entity['entity_id']}: {entity.get('name') or '(no name)'}")
        print(f"    Type: {entity['entity_type']}, Polymer: {entity.get('polymer_type')}")
        print(f"    Chains: {entity['chain_ids']}")
    
    print("\n--- Chains Summary ---")
    summary = result.get('summary', {})
    print(f"  Proteins: {summary.get('num_protein_chains', 0)} chains")
    print(f"  Ligands: {summary.get('num_ligand_chains', 0)} chains")
    
    print("\n--- Chain Details ---")
    for chain in result.get('chains', []):
        print(f"  Chain {chain['chain_id']}: {chain['chain_type']}")
        print(f"    Residues: {chain['num_residues']}, Atoms: {chain['num_atoms']}")
        print(f"    Residue types: {chain['residue_names']}")


2025-12-05 19:09:37,721 - servers.structure_server - INFO - Inspecting molecules in: boltz_results_ligand/predictions/ligand/ligand_model_0.cif


Test 2.2: Inspect Boltz-2 predicted structure


                    INFO     Inspecting molecules in:                                       ]8;id=341231;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=344148;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#872\872]8;;\
                             boltz_results_ligand/predictions/ligand/ligand_model_0.cif                            

2025-12-05 19:09:37,723 - servers.structure_server - INFO - Reading structure with gemmi (.cif)...


                    INFO     Reading structure with gemmi (.cif)...                         ]8;id=885380;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=523857;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#925\925]8;;\

2025-12-05 19:09:37,744 - servers.structure_server - INFO - Successfully inspected structure: 6 chains found


                    INFO     Successfully inspected structure: 6 chains found              ]8;id=730017;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=998639;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1116\1116]8;;\

2025-12-05 19:09:37,745 - servers.structure_server - INFO -   Proteins: 2, Ligands: 4, Waters: 0, Ions: 0


                    INFO       Proteins: 2, Ligands: 4, Waters: 0, Ions: 0                 ]8;id=599422;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=546058;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1117\1117]8;;\


 Inspect Boltz-2 Prediction

✓ SUCCESS

Details:
  source_file: boltz_results_ligand/predictions/ligand/ligand_model_0.cif
  file_format: cif
  header: {'pdb_id': 'model'}
  entities: [complex data, 3]
  num_models: 1
  chains: [complex data, 6]
  summary: [complex data, dict]

--- Header Information ---
  pdb_id: model

--- Entities ---
  Entity 1: (no name)
    Type: polymer, Polymer: PeptideL
    Chains: ['A', 'B']
  Entity 2: (no name)
    Type: nonpolymer, Polymer: None
    Chains: ['C', 'D']
  Entity 3: (no name)
    Type: nonpolymer, Polymer: None
    Chains: ['E', 'F']

--- Chains Summary ---
  Proteins: 2 chains
  Ligands: 4 chains

--- Chain Details ---
  Chain A: protein
    Residues: 384, Atoms: 2961
    Residue types: ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS', 'ILE', 'LEU', 'LYS', 'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL']
  Chain B: protein
    Residues: 384, Atoms: 2961
    Residue types: ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU'

---
## Test 3: split_molecules

Test splitting multi-chain structures into individual chain files.
The `split_molecules` function uses `inspect_molecules` internally.


In [51]:
# Test 3.1: Split 1AKE (homodimer with ligand)
print("Test 3.1: Split 1AKE structure")

# First fetch 1AKE (if not already available)
fetch_result = asyncio.run(fetch_molecules("1AKE", source="pdb"))
if fetch_result['success']:
    result = split_molecules(fetch_result['file_path'])
    show_result(result, "Split 1AKE")
    
    # Show chain files
    if result['success']:
        print("\nProtein files:")
        for f in result['protein_files']:
            print(f"  - {f}")
        print("\nLigand files:")
        for f in result['ligand_files']:
            print(f"  - {f}")
        if result['ion_files']:
            print("\nIon files:")
            for f in result['ion_files']:
                print(f"  - {f}")
        
        # Show chain mapping
        print("\nChain to file mapping:")
        for info in result.get('chain_file_info', []):
            print(f"  Chain {info['chain_id']} ({info['chain_type']}): {info['file']}")
else:
    print("Failed to fetch 1AKE for split test")


2025-12-05 19:09:37,752 - servers.structure_server - INFO - Fetching 1AKE from pdb


Test 3.1: Split 1AKE structure


                    INFO     Fetching 1AKE from pdb                                         ]8;id=256762;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=663924;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#675\675]8;;\

                    INFO     HTTP Request: GET https://files.rcsb.org/download/1AKE.cif "HTTP/1.1   ]8;id=189610;file:///Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=832003;file:///Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/httpx/_client.py#1740\1740]8;;\
                             200 OK"                                                                               

2025-12-05 19:09:37,787 - servers.structure_server - INFO - Downloaded 1AKE to output/1AKE.cif


...

2025-12-05 19:09:37,862 - servers.structure_server - INFO - Wrote ligand: output/85347c82/ligand_2.pdb


                    INFO     Wrote ligand: output/85347c82/ligand_2.pdb                    ]8;id=636123;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=363458;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1348\1348]8;;\

2025-12-05 19:09:37,864 - servers.structure_server - INFO - Successfully split structure: 2 protein, 2 ligand, 0 ion, 0 water files


                    INFO     Successfully split structure: 2 protein, 2 ligand, 0 ion, 0   ]8;id=65984;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=815616;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1375\1375]8;;\
                             water files                                                                           


 Split 1AKE

✓ SUCCESS

Details:
  job_id: 85347c82
  output_dir: output/85347c82
  source_file: output/1AKE.cif
  file_format: pdb
  protein_files: ['output/85347c82/protein_1.pdb', 'output/85347c82/protein_2.pdb']
  ligand_files: ['output/85347c82/ligand_1.pdb', 'output/85347c82/ligand_2.pdb']
  ion_files: []
  water_files: []
  all_chains: [complex data, 6]
  chain_file_info: [complex data, 4]
  exclude_waters: True

Protein files:
  - output/85347c82/protein_1.pdb
  - output/85347c82/protein_2.pdb

Ligand files:
  - output/85347c82/ligand_1.pdb
  - output/85347c82/ligand_2.pdb

Chain to file mapping:
  Chain A (protein): output/85347c82/protein_1.pdb
  Chain B (protein): output/85347c82/protein_2.pdb
  Chain C (ligand): output/85347c82/ligand_1.pdb
  Chain D (ligand): output/85347c82/ligand_2.pdb


In [52]:
# Test 3.2: Split with chain selection
print("Test 3.2: Split 1AKE - select only chain A")

if fetch_result['success']:
    result = split_molecules(
        fetch_result['file_path'],
        select_chains=['A']
    )
    show_result(result, "Split 1AKE (Chain A only)")
    
    if result['success']:
        print(f"\nExtracted {len(result['protein_files'])} protein chain(s)")
        print(f"Output directory: {result['output_dir']}")
else:
    print("Skipped - 1AKE not available")


2025-12-05 19:09:37,869 - servers.structure_server - INFO - Splitting structure: output/1AKE.cif


Test 3.2: Split 1AKE - select only chain A


                    INFO     Splitting structure: output/1AKE.cif                          ]8;id=692475;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=222932;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1174\1174]8;;\

2025-12-05 19:09:37,870 - servers.structure_server - INFO - Inspecting molecules in: output/1AKE.cif


                    INFO     Inspecting molecules in: output/1AKE.cif                       ]8;id=85077;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=450261;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#872\872]8;;\

...

2025-12-05 19:09:37,908 - servers.structure_server - INFO - Wrote protein: output/a8bb87f0/protein_1.pdb


                    INFO     Wrote protein: output/a8bb87f0/protein_1.pdb                  ]8;id=223224;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=627050;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1348\1348]8;;\

2025-12-05 19:09:37,909 - servers.structure_server - INFO - Successfully split structure: 1 protein, 0 ligand, 0 ion, 0 water files


                    INFO     Successfully split structure: 1 protein, 0 ligand, 0 ion, 0   ]8;id=622884;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=325515;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1375\1375]8;;\
                             water files                                                                           


 Split 1AKE (Chain A only)

✓ SUCCESS

Details:
  job_id: a8bb87f0
  output_dir: output/a8bb87f0
  source_file: output/1AKE.cif
  file_format: pdb
  protein_files: ['output/a8bb87f0/protein_1.pdb']
  ligand_files: []
  ion_files: []
  water_files: []
  all_chains: [complex data, 6]
  chain_file_info: [complex data, 1]
  exclude_waters: True

Extracted 1 protein chain(s)
Output directory: output/a8bb87f0


In [53]:
# Test 3.3: Split Boltz-2 predicted structure
print("Test 3.3: Split Boltz-2 predicted structure")

boltz_cif = "boltz_results_ligand/predictions/ligand/ligand_model_0.cif"
result = split_molecules(boltz_cif)
show_result(result, "Split Boltz-2 Prediction")

if result['success']:
    print("\nProtein files:")
    for f in result['protein_files']:
        print(f"  - {f}")
    print("\nLigand files:")
    for f in result['ligand_files']:
        print(f"  - {f}")
    if result['ion_files']:
        print("\nIon files:")
        for f in result['ion_files']:
            print(f"  - {f}")
    
    # Show chain mapping
    print("\nChain to file mapping:")
    for info in result.get('chain_file_info', []):
        print(f"  Chain {info['chain_id']} ({info['chain_type']}): {info['file']}")


2025-12-05 19:09:37,915 - servers.structure_server - INFO - Splitting structure: boltz_results_ligand/predictions/ligand/ligand_model_0.cif


Test 3.3: Split Boltz-2 predicted structure


                    INFO     Splitting structure:                                          ]8;id=946373;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=112789;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1174\1174]8;;\
                             boltz_results_ligand/predictions/ligand/ligand_model_0.cif                            

2025-12-05 19:09:37,916 - servers.structure_server - INFO - Inspecting molecules in: boltz_results_ligand/predictions/ligand/ligand_model_0.cif


                    INFO     Inspecting molecules in:                                       ]8;id=486409;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=73994;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#872\872]8;;\
                             boltz_results_ligand/predictions/ligand/ligand_model_0.cif                            

...

2025-12-05 19:09:38,007 - servers.structure_server - INFO - Wrote ligand: output/adec8515/ligand_4.pdb


                    INFO     Wrote ligand: output/adec8515/ligand_4.pdb                    ]8;id=160523;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=271370;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1348\1348]8;;\

2025-12-05 19:09:38,009 - servers.structure_server - INFO - Successfully split structure: 2 protein, 4 ligand, 0 ion, 0 water files


                    INFO     Successfully split structure: 2 protein, 4 ligand, 0 ion, 0   ]8;id=17594;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=525338;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1375\1375]8;;\
                             water files                                                                           


 Split Boltz-2 Prediction

✓ SUCCESS

Details:
  job_id: adec8515
  output_dir: output/adec8515
  source_file: boltz_results_ligand/predictions/ligand/ligand_model_0.cif
  file_format: pdb
  protein_files: ['output/adec8515/protein_1.pdb', 'output/adec8515/protein_2.pdb']
  ligand_files: [complex data, 4]
  ion_files: []
  water_files: []
  all_chains: [complex data, 6]
  chain_file_info: [complex data, 6]
  exclude_waters: True

Protein files:
  - output/adec8515/protein_1.pdb
  - output/adec8515/protein_2.pdb

Ligand files:
  - output/adec8515/ligand_1.pdb
  - output/adec8515/ligand_2.pdb
  - output/adec8515/ligand_3.pdb
  - output/adec8515/ligand_4.pdb

Chain to file mapping:
  Chain A (protein): output/adec8515/protein_1.pdb
  Chain B (protein): output/adec8515/protein_2.pdb
  Chain C (ligand): output/adec8515/ligand_1.pdb
  Chain D (ligand): output/adec8515/ligand_2.pdb
  Chain E (ligand): output/adec8515/ligand_3.pdb
  Chain F (ligand): output/adec8515/ligand_4.pdb


---
## Test 4: clean_protein

Test protein structure cleaning with PDBFixer.


In [54]:
# Test 4.1: Clean 1CRN (crambin - has disulfide bonds)
print("Test 4.1: Clean 1CRN (crambin with disulfide bonds)")

# First fetch and split
fetch_result = asyncio.run(fetch_molecules("1CRN", source="pdb"))
if fetch_result['success']:
    split_result = split_molecules(fetch_result['file_path'])
    if split_result['success'] and split_result['protein_files']:
        protein_pdb = split_result['protein_files'][0]
        
        result = clean_protein(protein_pdb)
        show_result(result, "Clean 1CRN")
        
        # Check disulfide bonds
        if result.get('disulfide_bonds'):
            print("\nDisulfide bonds detected:")
            for bond in result['disulfide_bonds']:
                print(f"  {bond['residue1']} <-> {bond['residue2']}")
    else:
        print("Failed to split 1CRN")
else:
    print("Failed to fetch 1CRN")


2025-12-05 19:09:38,015 - servers.structure_server - INFO - Fetching 1CRN from pdb


Test 4.1: Clean 1CRN (crambin with disulfide bonds)


                    INFO     Fetching 1CRN from pdb                                         ]8;id=260119;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=16930;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#675\675]8;;\

                    INFO     HTTP Request: GET https://files.rcsb.org/download/1CRN.cif "HTTP/1.1   ]8;id=378471;file:///Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=861706;file:///Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/httpx/_client.py#1740\1740]8;;\
                             200 OK"                                                                               

2025-12-05 19:09:38,043 - servers.structure_server - INFO - Downloaded 1CRN to output/1CRN.cif


...

2025-12-05 19:09:39,553 - servers.structure_server - INFO - pdb4amber conversion successful: output/d0b00ed3/protein_1.amber.pdb


[12/05/25 19:09:39] INFO     pdb4amber conversion successful:                              ]8;id=41369;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=410895;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1775\1775]8;;\
                             output/d0b00ed3/protein_1.amber.pdb                                                   

2025-12-05 19:09:39,555 - servers.structure_server - INFO - Successfully cleaned protein structure: output/d0b00ed3/protein_1.amber.pdb


                    INFO     Successfully cleaned protein structure:                       ]8;id=42259;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=145715;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1796\1796]8;;\
                             output/d0b00ed3/protein_1.amber.pdb                                                   


 Clean 1CRN

✓ SUCCESS

Details:
  output_file: output/d0b00ed3/protein_1.amber.pdb
  input_file: output/d0b00ed3/protein_1.pdb
  cap_termini_required: False
  statistics: {'initial_chains': 1, 'initial_residues': 46, 'final_residues': 46, 'final_atoms': 618}
  disulfide_bonds: [complex data, 3]
  pdbfixer_output: output/d0b00ed3/protein_1.clean.pdb

Operations:
  ✓ load_structure: success - Loaded 1 chain(s), 46 residue(s)
  ○ missing_residues: none_found - No missing residues found
  ○ nonstandard_residues: none_found - No non-standard residues found
  ✓ remove_heterogens: success - Removed heterogens, water removed
  ○ missing_atoms: none_found - No missing atoms or residues found
  ✓ disulfide_bonds: detected - Found 3 disulfide bond(s), renamed 6 CYS -> CYX for Amber
  ✓ protonation: success - Added hydrogens at pH 7.4
  ✓ write_output: success - Wrote 618 atoms to output/d0b00ed3/protein_1.clean.pdb
  ✓ pdb4amber: success - Converted to Amber conventions: output/d0b00ed3/protein

In [55]:
# Test 4.2: Clean with custom options
print("Test 4.2: Clean with custom options (with termini capping)")

if fetch_result['success'] and split_result['success']:
    protein_pdb = split_result['protein_files'][0]
    
    result = clean_protein(
        protein_pdb,
        cap_termini=True,
        ph=7.0
    )
    show_result(result, "Clean with Custom Options")
else:
    print("Skipped - previous test failed")


2025-12-05 19:09:39,561 - servers.structure_server - INFO - Cleaning protein structure: output/d0b00ed3/protein_1.pdb


Test 4.2: Clean with custom options (with termini capping)


                    INFO     Cleaning protein structure: output/d0b00ed3/protein_1.pdb     ]8;id=598000;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=80517;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1436\1436]8;;\

2025-12-05 19:09:39,563 - servers.structure_server - INFO - Loading structure with PDBFixer


                    INFO     Loading structure with PDBFixer                               ]8;id=131810;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=940470;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1465\1465]8;;\

...

2025-12-05 19:09:40,830 - servers.structure_server - INFO - pdb4amber conversion successful: output/d0b00ed3/protein_1.amber.pdb


[12/05/25 19:09:40] INFO     pdb4amber conversion successful:                              ]8;id=361886;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=723579;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1775\1775]8;;\
                             output/d0b00ed3/protein_1.amber.pdb                                                   

2025-12-05 19:09:40,832 - servers.structure_server - INFO - Successfully cleaned protein structure: output/d0b00ed3/protein_1.amber.pdb


                    INFO     Successfully cleaned protein structure:                       ]8;id=529370;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=759580;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1796\1796]8;;\
                             output/d0b00ed3/protein_1.amber.pdb                                                   


 Clean with Custom Options

✓ SUCCESS

Details:
  output_file: output/d0b00ed3/protein_1.amber.pdb
  input_file: output/d0b00ed3/protein_1.pdb
  cap_termini_required: True
  statistics: {'initial_chains': 1, 'initial_residues': 46, 'final_residues': 48, 'final_atoms': 627}
  disulfide_bonds: [complex data, 3]
  pdbfixer_output: output/d0b00ed3/protein_1.clean.pdb

Operations:
  ✓ load_structure: success - Loaded 1 chain(s), 46 residue(s)
  ○ terminal_caps: added_to_missing - Added ACE/NME caps as missing residues for 1 chain(s): ['A']
  ○ nonstandard_residues: none_found - No non-standard residues found
  ✓ remove_heterogens: success - Removed heterogens, water removed
  ✓ missing_atoms: added - Added 2 missing residue(s)
  ✓ disulfide_bonds: detected - Found 3 disulfide bond(s), renamed 6 CYS -> CYX for Amber
  ✓ protonation: success - Added hydrogens at pH 7.0
  ✓ write_output: success - Wrote 627 atoms to output/d0b00ed3/protein_1.clean.pdb
  ✓ pdb4amber: success - Converted to Amb

In [56]:
# Test 4.3: Clean non-existent file (error handling)
print("Test 4.3: Clean non-existent file")

result = clean_protein("/nonexistent/protein.pdb")
show_result(result, "Clean Non-existent File")

assert not result['success'], "Should fail for non-existent file"
print("\n✓ File not found error handling works")


2025-12-05 19:09:40,838 - servers.structure_server - INFO - Cleaning protein structure: /nonexistent/protein.pdb


Test 4.3: Clean non-existent file


                    INFO     Cleaning protein structure: /nonexistent/protein.pdb          ]8;id=632141;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=211502;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1436\1436]8;;\

2025-12-05 19:09:40,840 - servers.structure_server - ERROR - Input file not found: /nonexistent/protein.pdb


                    ERROR    Input file not found: /nonexistent/protein.pdb                ]8;id=574283;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=415607;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1455\1455]8;;\


 Clean Non-existent File

✗ FAILED

Errors:
  - Input file not found: /nonexistent/protein.pdb

Details:
  output_file: None
  input_file: /nonexistent/protein.pdb
  cap_termini_required: False
  statistics: {}
  disulfide_bonds: []

✓ File not found error handling works


---
## Test 5: clean_ligand

Test ligand cleaning using SMILES template matching.


In [57]:
# Test 5.1: Clean ligand from 1AKE (AP5A inhibitor)
print("Test 5.1: Clean ligand from 1AKE")

# Fetch and split 1AKE to get ligand
fetch_result = asyncio.run(fetch_molecules("1AKE", source="pdb"))
if fetch_result['success']:
    # Split to get ligand chains
    split_result = split_molecules(fetch_result['file_path'])
    
    if split_result['success'] and split_result['ligand_files']:
        ligand_pdb = split_result['ligand_files'][0]
        print(f"Ligand PDB: {ligand_pdb}")
        
        # Get ligand ID from chain info
        ligand_info = [c for c in split_result['chain_file_info'] if c['chain_type'] == 'ligand'][0]
        ligand_id = split_result['all_chains'][2]['residue_names'][0]  # Get ligand name
        print(f"Ligand ID: {ligand_id}")
        
        # Clean ligand using SMILES template matching
        result = clean_ligand(
            ligand_pdb=ligand_pdb,
            ligand_id=ligand_id,  # AP5A
            target_ph=7.4,
            optimize=True
        )
        show_result(result, "Clean 1AKE Ligand (AP5A)")
        
        if result['success']:
            print(f"\nOutput SDF: {result['sdf_file']}")
            print(f"Net charge: {result['net_charge']}")
            print(f"SMILES source: {result['smiles_source']}")
    else:
        print("No ligand files found")
else:
    print("Failed to fetch 1AKE")


2025-12-05 19:09:40,851 - servers.structure_server - INFO - Fetching 1AKE from pdb


Test 5.1: Clean ligand from 1AKE


                    INFO     Fetching 1AKE from pdb                                         ]8;id=699200;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=942876;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#675\675]8;;\

                    INFO     HTTP Request: GET https://files.rcsb.org/download/1AKE.cif "HTTP/1.1   ]8;id=798592;file:///Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=741073;file:///Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/httpx/_client.py#1740\1740]8;;\
                             200 OK"                                                                               

2025-12-05 19:09:40,890 - servers.structure_server - INFO - Downloaded 1AKE to output/1AKE.cif


...

2025-12-05 19:09:41,819 - servers.structure_server - INFO - Wrote prepared ligand: /Users/yasu/tmp/mcp-md/notebooks/output/0ca00649/ligand_1_prepared.sdf


                    INFO     Wrote prepared ligand:                                        ]8;id=186411;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=519511;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#2192\2192]8;;\
                             /Users/yasu/tmp/mcp-md/notebooks/output/0ca00649/ligand_1_pre                         
                             pared.sdf                                                                             

2025-12-05 19:09:41,820 - servers.structure_server - INFO - Successfully cleaned ligand: /Users/yasu/tmp/mcp-md/notebooks/output/0ca00649/ligand_1_prepared.sdf


                    INFO     Successfully cleaned ligand:                                  ]8;id=77315;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=323890;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#2205\2205]8;;\
                             /Users/yasu/tmp/mcp-md/notebooks/output/0ca00649/ligand_1_pre                         
                             pared.sdf                                                                             


 Clean 1AKE Ligand (AP5A)

✓ SUCCESS

Warnings:
  - Template matching with H failed: Template matching failed: No matching found. PDB atoms: 57, Template atoms: 81, trying without H

Details:
  ligand_pdb: output/0ca00649/ligand_1.pdb
  ligand_id: AP5
  sdf_file: /Users/yasu/tmp/mcp-md/notebooks/output/0ca00649/ligand_1_prepared.sdf
  net_charge: -5
  charge_source: dimorphite
  mol_formal_charge: -5
  smiles_used: Nc1ncnc2c1ncn2[C@@H]1O[C@H](CO[P@](=O)([O-])O[P@](=O)([O-])OP(=O)([O-])O[P@@](=O)([O-])O[P@@](=O)([O-])OC[C@H]2O[C@@H](n3cnc4c(N)ncnc43)[C@H](O)[C@@H]2O)[C@@H](O)[C@H]1O
  smiles_original: c1nc(c2c(n1)n(cn2)[C@H]3[C@@H]([C@@H]([C@H](O3)CO[P@](=O)(O)O[P@](=O)(O)OP(=O)(O)O[P@@](=O)(O)O[P@@](=O)(O)OC[C@@H]4[C@H]([C@H]([C@@H](O4)n5cnc6c5ncnc6N)O)O)O)O)N
  smiles_source: ccd
  target_ph: 7.4
  num_atoms: 81
  num_heavy_atoms: 57
  optimized: True
  optimization_converged: False
  output_dir: /Users/yasu/tmp/mcp-md/notebooks/output/0ca00649

Output SDF: /Users/yasu/tmp/mcp-md/not

In [58]:
# Test 5.2: Clean SAH ligand from Boltz-2 prediction
print("Test 5.2: Clean SAH ligand from Boltz-2 prediction")

boltz_cif = "boltz_results_ligand/predictions/ligand/ligand_model_0.cif"
split_result = split_molecules(boltz_cif)

if split_result['success'] and split_result['ligand_files']:
    # Find SAH ligand chain
    sah_file = None
    sah_chain = None
    for info in split_result['chain_file_info']:
        if info['chain_type'] == 'ligand':
            # Get residue name from all_chains
            for chain in split_result['all_chains']:
                if chain['chain_id'] == info['chain_id']:
                    if 'SAH' in chain['residue_names']:
                        sah_file = info['file']
                        sah_chain = chain
                        break
        if sah_file:
            break
    
    if sah_file:
        print(f"SAH ligand PDB: {sah_file}")
        
        result = clean_ligand(
            ligand_pdb=sah_file,
            ligand_id="SAH",
            target_ph=7.4,
            optimize=True
        )
        show_result(result, "Clean Boltz-2 SAH Ligand")
        
        if result['success']:
            print(f"\nOutput SDF: {result['sdf_file']}")
            print(f"Net charge: {result['net_charge']}")
    else:
        print("SAH ligand not found in Boltz-2 structure")
else:
    print("Failed to split Boltz-2 structure")


2025-12-05 19:09:41,827 - servers.structure_server - INFO - Splitting structure: boltz_results_ligand/predictions/ligand/ligand_model_0.cif


Test 5.2: Clean SAH ligand from Boltz-2 prediction


                    INFO     Splitting structure:                                          ]8;id=982223;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=101176;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1174\1174]8;;\
                             boltz_results_ligand/predictions/ligand/ligand_model_0.cif                            

2025-12-05 19:09:41,828 - servers.structure_server - INFO - Inspecting molecules in: boltz_results_ligand/predictions/ligand/ligand_model_0.cif


                    INFO     Inspecting molecules in:                                       ]8;id=61505;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=707572;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#872\872]8;;\
                             boltz_results_ligand/predictions/ligand/ligand_model_0.cif                            

...

2025-12-05 19:09:42,671 - servers.structure_server - INFO - Wrote prepared ligand: /Users/yasu/tmp/mcp-md/notebooks/output/3463ffd6/ligand_1_prepared.sdf


                    INFO     Wrote prepared ligand:                                        ]8;id=298412;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=833648;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#2192\2192]8;;\
                             /Users/yasu/tmp/mcp-md/notebooks/output/3463ffd6/ligand_1_pre                         
                             pared.sdf                                                                             

2025-12-05 19:09:42,674 - servers.structure_server - INFO - Successfully cleaned ligand: /Users/yasu/tmp/mcp-md/notebooks/output/3463ffd6/ligand_1_prepared.sdf


                    INFO     Successfully cleaned ligand:                                  ]8;id=952590;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=939523;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#2205\2205]8;;\
                             /Users/yasu/tmp/mcp-md/notebooks/output/3463ffd6/ligand_1_pre                         
                             pared.sdf                                                                             


 Clean Boltz-2 SAH Ligand

✓ SUCCESS

Warnings:
  - Template matching with H failed: Template matching failed: No matching found. PDB atoms: 26, Template atoms: 45, trying without H

Details:
  ligand_pdb: output/3463ffd6/ligand_1.pdb
  ligand_id: SAH
  sdf_file: /Users/yasu/tmp/mcp-md/notebooks/output/3463ffd6/ligand_1_prepared.sdf
  net_charge: -1
  charge_source: dimorphite
  mol_formal_charge: -1
  smiles_used: Nc1ncnc2c1ncn2[C@@H]1O[C@H](CSCC[C@H](N)C(=O)[O-])[C@@H](O)[C@H]1O
  smiles_original: c1nc(c2c(n1)n(cn2)[C@H]3[C@@H]([C@@H]([C@H](O3)CSCC[C@@H](C(=O)O)N)O)O)N
  smiles_source: ccd
  target_ph: 7.4
  num_atoms: 45
  num_heavy_atoms: 26
  optimized: True
  optimization_converged: False
  output_dir: /Users/yasu/tmp/mcp-md/notebooks/output/3463ffd6

Output SDF: /Users/yasu/tmp/mcp-md/notebooks/output/3463ffd6/ligand_1_prepared.sdf
Net charge: -1


In [59]:
# Test 5.3: Clean ligand with user-provided SMILES
print("Test 5.3: Clean ligand with user-provided SMILES")

# Use Boltz-2 SAH ligand with explicit SMILES
boltz_cif = "boltz_results_ligand/predictions/ligand/ligand_model_0.cif"
split_result = split_molecules(boltz_cif)

if split_result['success'] and split_result['ligand_files']:
    # Get first ligand file
    ligand_file = split_result['ligand_files'][0]
    ligand_chain = split_result['chain_file_info'][2]  # First ligand
    ligand_name = split_result['all_chains'][2]['residue_names'][0]
    
    print(f"Ligand: {ligand_name}")
    print(f"File: {ligand_file}")
    
    # SAH SMILES from PDB CCD
    sah_smiles = "Nc1ncnc2c1ncn2[C@@H]1O[C@H](CSCC[C@H](N)C(=O)O)[C@@H](O)[C@H]1O"
    
    result = clean_ligand(
        ligand_pdb=ligand_file,
        ligand_id=ligand_name,
        smiles=sah_smiles,  # User-provided SMILES
        target_ph=7.4,
        optimize=False  # Skip optimization for speed
    )
    show_result(result, "Clean Ligand with User SMILES")
    
    if result['success']:
        print(f"\nSMILES source: {result['smiles_source']}")  # Should be 'user'
else:
    print("Failed to get ligand from Boltz-2")


2025-12-05 19:09:42,682 - servers.structure_server - INFO - Splitting structure: boltz_results_ligand/predictions/ligand/ligand_model_0.cif


Test 5.3: Clean ligand with user-provided SMILES


                    INFO     Splitting structure:                                          ]8;id=460358;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=645854;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1174\1174]8;;\
                             boltz_results_ligand/predictions/ligand/ligand_model_0.cif                            

2025-12-05 19:09:42,685 - servers.structure_server - INFO - Inspecting molecules in: boltz_results_ligand/predictions/ligand/ligand_model_0.cif


                    INFO     Inspecting molecules in:                                       ]8;id=378003;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=804145;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#872\872]8;;\
                             boltz_results_ligand/predictions/ligand/ligand_model_0.cif                            

...

2025-12-05 19:09:42,787 - servers.structure_server - INFO - Wrote prepared ligand: /Users/yasu/tmp/mcp-md/notebooks/output/b09df635/ligand_1_prepared.sdf


                    INFO     Wrote prepared ligand:                                        ]8;id=953305;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=428227;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#2192\2192]8;;\
                             /Users/yasu/tmp/mcp-md/notebooks/output/b09df635/ligand_1_pre                         
                             pared.sdf                                                                             

2025-12-05 19:09:42,788 - servers.structure_server - INFO - Successfully cleaned ligand: /Users/yasu/tmp/mcp-md/notebooks/output/b09df635/ligand_1_prepared.sdf


                    INFO     Successfully cleaned ligand:                                  ]8;id=177851;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=861738;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#2205\2205]8;;\
                             /Users/yasu/tmp/mcp-md/notebooks/output/b09df635/ligand_1_pre                         
                             pared.sdf                                                                             


 Clean Ligand with User SMILES

✓ SUCCESS

Warnings:
  - Template matching with H failed: Template matching failed: No matching found. PDB atoms: 26, Template atoms: 45, trying without H

Details:
  ligand_pdb: output/b09df635/ligand_1.pdb
  ligand_id: SAH
  sdf_file: /Users/yasu/tmp/mcp-md/notebooks/output/b09df635/ligand_1_prepared.sdf
  net_charge: -1
  charge_source: dimorphite
  mol_formal_charge: -1
  smiles_used: Nc1ncnc2c1ncn2[C@@H]1O[C@H](CSCC[C@H](N)C(=O)[O-])[C@@H](O)[C@H]1O
  smiles_original: Nc1ncnc2c1ncn2[C@@H]1O[C@H](CSCC[C@H](N)C(=O)O)[C@@H](O)[C@H]1O
  smiles_source: user
  target_ph: 7.4
  num_atoms: 45
  num_heavy_atoms: 26
  optimized: False
  optimization_converged: False
  output_dir: /Users/yasu/tmp/mcp-md/notebooks/output/b09df635

SMILES source: user


---
## Test 6: run_antechamber_robust

Test GAFF2 parameterization with AM1-BCC charges.


In [60]:
# Test 6.1: Run antechamber on cleaned SAH ligand
print("Test 6.1: Run antechamber on SAH ligand (GAFF2 + AM1-BCC)")

# First clean the SAH ligand
boltz_cif = "boltz_results_ligand/predictions/ligand/ligand_model_0.cif"
split_result = split_molecules(boltz_cif)

if split_result['success'] and split_result['ligand_files']:
    # Find SAH ligand
    sah_file = None
    for info in split_result['chain_file_info']:
        if info['chain_type'] == 'ligand':
            for chain in split_result['all_chains']:
                if chain['chain_id'] == info['chain_id'] and 'SAH' in chain['residue_names']:
                    sah_file = info['file']
                    break
        if sah_file:
            break
    
    if sah_file:
        # Clean ligand first
        clean_result = clean_ligand(
            ligand_pdb=sah_file,
            ligand_id="SAH",
            target_ph=7.4
        )
        
        if clean_result['success']:
            print(f"Cleaned SDF: {clean_result['sdf_file']}")
            print(f"Net charge: {clean_result['net_charge']}")
            
            # Run antechamber
            result = run_antechamber_robust(
                ligand_file=clean_result['sdf_file'],
                net_charge=clean_result['net_charge'],
                residue_name="SAH"
            )
            show_result(result, "Antechamber SAH")
            
            if result['success']:
                print(f"\nGenerated files:")
                print(f"  MOL2: {result['mol2']}")
                print(f"  FRCMOD: {result['frcmod']}")
                print(f"  Total charge: {result['total_charge']:.4f}")
                
                # Check frcmod validation
                if result['frcmod_validation']:
                    if result['frcmod_validation']['valid']:
                        print("  frcmod: ✓ Valid")
                    else:
                        print(f"  frcmod: ✗ {result['frcmod_validation']['attn_count']} parameters need attention")
        else:
            print(f"Clean failed: {clean_result['errors']}")
    else:
        print("SAH not found")
else:
    print("Failed to split structure")


2025-12-05 19:09:42,794 - servers.structure_server - INFO - Splitting structure: boltz_results_ligand/predictions/ligand/ligand_model_0.cif


Test 6.1: Run antechamber on SAH ligand (GAFF2 + AM1-BCC)


                    INFO     Splitting structure:                                          ]8;id=347909;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=367021;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1174\1174]8;;\
                             boltz_results_ligand/predictions/ligand/ligand_model_0.cif                            

2025-12-05 19:09:42,795 - servers.structure_server - INFO - Inspecting molecules in: boltz_results_ligand/predictions/ligand/ligand_model_0.cif


                    INFO     Inspecting molecules in:                                       ]8;id=727419;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=77937;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#872\872]8;;\
                             boltz_results_ligand/predictions/ligand/ligand_model_0.cif                            

...

2025-12-05 19:10:17,860 - servers.structure_server - INFO - parmchk2 completed: /Users/yasu/tmp/mcp-md/notebooks/output/2fdaadfa/ligand_1_prepared.frcmod


                    INFO     parmchk2 completed:                                           ]8;id=443003;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=545557;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#2446\2446]8;;\
                             /Users/yasu/tmp/mcp-md/notebooks/output/2fdaadfa/ligand_1_pre                         
                             pared.frcmod                                                                          

2025-12-05 19:10:17,864 - servers.structure_server - INFO - Successfully parameterized ligand: /Users/yasu/tmp/mcp-md/notebooks/output/2fdaadfa/ligand_1_prepared.gaff.mol2


                    INFO     Successfully parameterized ligand:                            ]8;id=640451;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=400304;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#2495\2495]8;;\
                             /Users/yasu/tmp/mcp-md/notebooks/output/2fdaadfa/ligand_1_pre                         
                             pared.gaff.mol2                                                                       


 Antechamber SAH

✓ SUCCESS

Details:
  mol2: /Users/yasu/tmp/mcp-md/notebooks/output/2fdaadfa/ligand_1_prepared.gaff.mol2
  frcmod: /Users/yasu/tmp/mcp-md/notebooks/output/2fdaadfa/ligand_1_prepared.frcmod
  charge_used: -1
  charge_method: bcc
  atom_type: gaff2
  residue_name: SAH
  charges: [complex data, 45]
  total_charge: -1.0000000000000016
  frcmod_validation: [complex data, dict]
  sqm_diagnostics: None
  charge_estimation: None
  diagnostics_dir: /Users/yasu/tmp/mcp-md/notebooks/output/2fdaadfa/diagnostics

Generated files:
  MOL2: /Users/yasu/tmp/mcp-md/notebooks/output/2fdaadfa/ligand_1_prepared.gaff.mol2
  FRCMOD: /Users/yasu/tmp/mcp-md/notebooks/output/2fdaadfa/ligand_1_prepared.frcmod
  Total charge: -1.0000
  frcmod: ✓ Valid


In [61]:
# Test 6.2: Run antechamber with auto charge estimation
print("Test 6.2: Run antechamber with auto charge estimation")

# Use the same cleaned SDF but let antechamber estimate the charge
if 'clean_result' in dir() and clean_result['success']:
    result = run_antechamber_robust(
        ligand_file=clean_result['sdf_file'],
        net_charge=None,  # Auto-estimate
        residue_name="SAH",
        charge_method="bcc",
        atom_type="gaff2"
    )
    show_result(result, "Antechamber (Auto Charge)")
    
    if result['success']:
        print(f"\nCharge estimation:")
        if result['charge_estimation']:
            print(f"  Estimated: {result['charge_estimation'].get('estimated_charge_at_ph')}")
            print(f"  Confidence: {result['charge_estimation'].get('confidence')}")
        print(f"  Charge used: {result['charge_used']}")
else:
    print("Skipped - clean_result not available")


2025-12-05 19:10:17,869 - servers.structure_server - INFO - Running robust antechamber: /Users/yasu/tmp/mcp-md/notebooks/output/2fdaadfa/ligand_1_prepared.sdf


Test 6.2: Run antechamber with auto charge estimation


                    INFO     Running robust antechamber:                                   ]8;id=528932;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=997140;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#2262\2262]8;;\
                             /Users/yasu/tmp/mcp-md/notebooks/output/2fdaadfa/ligand_1_pre                         
                             pared.sdf                                                                             

2025-12-05 19:10:17,871 - servers.structure_server - INFO - Auto-estimating net charge...


                    INFO     Auto-estimating net charge...                                 ]8;id=976198;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=36921;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#2303\2303]8;;\

...

2025-12-05 19:10:52,306 - servers.structure_server - INFO - parmchk2 completed: /Users/yasu/tmp/mcp-md/notebooks/output/2fdaadfa/ligand_1_prepared.frcmod


[12/05/25 19:10:52] INFO     parmchk2 completed:                                           ]8;id=792624;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=308815;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#2446\2446]8;;\
                             /Users/yasu/tmp/mcp-md/notebooks/output/2fdaadfa/ligand_1_pre                         
                             pared.frcmod                                                                          

2025-12-05 19:10:52,310 - servers.structure_server - INFO - Successfully parameterized ligand: /Users/yasu/tmp/mcp-md/notebooks/output/2fdaadfa/ligand_1_prepared.gaff.mol2


                    INFO     Successfully parameterized ligand:                            ]8;id=874805;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=244922;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#2495\2495]8;;\
                             /Users/yasu/tmp/mcp-md/notebooks/output/2fdaadfa/ligand_1_pre                         
                             pared.gaff.mol2                                                                       


 Antechamber (Auto Charge)

✓ SUCCESS

Details:
  mol2: /Users/yasu/tmp/mcp-md/notebooks/output/2fdaadfa/ligand_1_prepared.gaff.mol2
  frcmod: /Users/yasu/tmp/mcp-md/notebooks/output/2fdaadfa/ligand_1_prepared.frcmod
  charge_used: -1
  charge_method: bcc
  atom_type: gaff2
  residue_name: SAH
  charges: [complex data, 45]
  total_charge: -1.0000000000000016
  frcmod_validation: [complex data, dict]
  sqm_diagnostics: None
  charge_estimation: [complex data, dict]
  diagnostics_dir: /Users/yasu/tmp/mcp-md/notebooks/output/2fdaadfa/diagnostics

Charge estimation:
  Estimated: -1
  Confidence: high
  Charge used: -1


---
## Test 7: Integration Test

Test complete workflows: fetch -> split -> clean_protein + clean_ligand -> antechamber


In [62]:
# Test 7.1: Complete workflow using prepare_complex
print("Test 7.1: Complete Boltz-2 workflow using prepare_complex")
print("="*60)

boltz_cif = "boltz_results_ligand/predictions/ligand/ligand_model_0.cif"

# Run complete workflow with a single function call
result = prepare_complex(
    structure_file=boltz_cif,
    ph=7.4,
    cap_termini=False,
    process_proteins=True,
    process_ligands=True,
    run_parameterization=True,
    optimize_ligands=True,
    # Optional: provide SMILES for specific ligands
    # ligand_smiles={"SAH": "Nc1ncnc2c1ncn2[C@@H]1O[C@H](CSCC[C@H](N)C(=O)O)[C@@H](O)[C@H]1O"}
)

show_result(result, "prepare_complex Result")

if result['success']:
    print("\n--- Summary ---")
    print(f"Output directory: {result['output_dir']}")
    
    # Show inspection summary
    if result['inspection']:
        summary = result['inspection'].get('summary', {})
        print(f"\nStructure: {summary.get('num_protein_chains', 0)} proteins, "
              f"{summary.get('num_ligand_chains', 0)} ligands")
    
    # Show processed proteins
    print(f"\nProteins processed: {len(result['proteins'])}")
    for p in result['proteins']:
        status = "✓" if p['success'] else "✗"
        print(f"  {status} Chain {p['chain_id']}: {p.get('output_file', 'N/A')}")
        if p['success'] and p.get('statistics'):
            print(f"      Atoms: {p['statistics'].get('final_atoms', 'N/A')}")
    
    # Show processed ligands
    print(f"\nLigands processed: {len(result['ligands'])}")
    for l in result['ligands']:
        status = "✓" if l['success'] else "✗"
        print(f"  {status} {l['ligand_id']} (Chain {l['chain_id']})")
        if l['success']:
            print(f"      SDF: {l.get('sdf_file', 'N/A')}")
            print(f"      MOL2: {l.get('mol2_file', 'N/A')}")
            print(f"      Charge: {l.get('net_charge', 'N/A')}")

print("\n" + "="*60)
print("Workflow complete!")


2025-12-05 19:10:52,316 - servers.structure_server - INFO - Preparing complex: boltz_results_ligand/predictions/ligand/ligand_model_0.cif


Test 7.1: Complete Boltz-2 workflow using prepare_complex


                    INFO     Preparing complex:                                            ]8;id=623904;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=792049;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#2587\2587]8;;\
                             boltz_results_ligand/predictions/ligand/ligand_model_0.cif                            

2025-12-05 19:10:52,318 - servers.structure_server - INFO - Step 1: Inspecting structure...


                    INFO     Step 1: Inspecting structure...                               ]8;id=646096;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=163722;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#2621\2621]8;;\

...

2025-12-05 19:12:03,229 - servers.structure_server - INFO -   Proteins processed: 2/2


                    INFO       Proteins processed: 2/2                                     ]8;id=323984;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=752160;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#2821\2821]8;;\

2025-12-05 19:12:03,230 - servers.structure_server - INFO -   Ligands processed: 2/4


                    INFO       Ligands processed: 2/4                                      ]8;id=842719;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=551760;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#2822\2822]8;;\


 prepare_complex Result

✓ SUCCESS

Warnings:
  - Ligand LIG1 cleaning failed: ['No SMILES found for ligand LIG1', "Hint: Provide SMILES manually via the 'smiles' parameter, or add it to KNOWN_LIGAND_SMILES dictionary"]
  - Ligand LIG1 cleaning failed: ['No SMILES found for ligand LIG1', "Hint: Provide SMILES manually via the 'smiles' parameter, or add it to KNOWN_LIGAND_SMILES dictionary"]

Details:
  job_id: 322818b5
  output_dir: output/8399c47c
  source_file: boltz_results_ligand/predictions/ligand/ligand_model_0.cif
  inspection: [complex data, dict]
  split: [complex data, dict]
  proteins: [complex data, 2]
  ligands: [complex data, 4]

--- Summary ---
Output directory: output/8399c47c

Structure: 2 proteins, 4 ligands

Proteins processed: 2
  ✓ Chain A: output/8399c47c/protein_1.amber.pdb
      Atoms: 5848
  ✓ Chain B: output/8399c47c/protein_2.amber.pdb
      Atoms: 5848

Ligands processed: 4
  ✓ SAH (Chain C)
      SDF: /Users/yasu/tmp/mcp-md/notebooks/output/8399c47c/ligand

---
## Summary

This notebook tested all tools in `structure_server.py`:

| Tool | Tests | Purpose |
|------|-------|---------|
| `fetch_molecules` | 2 | Download structures from PDB |
| `inspect_molecules` | 2 | Inspect structure files and analyze chains/molecules |
| `split_molecules` | 3 | Split multi-chain structures (including Boltz-2) |
| `clean_protein` | 3 | Clean and prepare proteins for MD |
| `clean_ligand` | 3 | Clean ligands using SMILES template matching |
| `run_antechamber_robust` | 2 | GAFF2 parameterization with AM1-BCC charges |
| `prepare_complex` | 1 | Complete workflow (split + clean + parameterize) |

### Key Features Tested:
- **LLM-friendly error handling**: All tools return structured `success`/`errors`/`warnings` fields
- **SMILES template matching**: Correct bond orders from CCD or user-provided SMILES
- **pH-dependent protonation**: Dimorphite-DL for correct protonation state
- **GAFF2 parameterization**: AM1-BCC charges with robust error handling
- **frcmod validation**: Check for missing/estimated parameters
- **Boltz-2 support**: Full workflow for AI-predicted protein-ligand complexes
- **One-step workflow**: `prepare_complex` combines all steps for convenience


In [63]:
print("All tests completed!")


All tests completed!
